# QUANTUM ESPRESSO Example App

**Author: Aliaksandr Yakutovich (LSMO/THEOS, EPFL)**

This apps allows to compute energy, band structure or to optimize the geometry of a material with a minimal set of input parameters.

It is powered by:
- [Quantum ESPRESSO](https://www.quantum-espresso.org/) as the quantum engine
- [AiiDA](http://www.aiida.net) as the automation platform
- [AiiDA-QUANTUMESPRESSO](https://github.com/aiidateam/aiida-quantumespresso) plugin including custom-made workflowsfor AiiDA to manage the selection of parameters, the error handling, etc.

### Example steps to run:
1. SCF
1. Relaxation
1. Band structure calculation

In [ ]:
%aiida

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports 
import os
import sys
from time import sleep 
import ipywidgets as ipw

# AiiDA imports
from aiida.orm import KpointsData
from aiida.engine import submit

# AiiDA lab imports
from aiidalab_widgets_base import viewer, CodeDropdown, CodQueryWidget, ExportButtonWidget
from aiidalab_widgets_base import ProcessFollowerWidget, ProgressBarWidget, RunningCalcJobOutputWidget, StructureUploadWidget
from aiidalab_widgets_base import StructureManagerWidget, SubmitButtonWidget, StructureBrowserWidget, SmilesWidget, StructureExamplesWidget

In [ ]:
arrow_down = ipw.HTML("""<hr>
                         <br />
                         <center>
                         <i class="fa fa-arrow-down" style="color:#B0B0B0;font-size:12em;"></i>
                         </center>
                        """)
hr = ipw.HTML('<hr>')
run_btn = ipw.Button(description='Submit calculation')
code_group = CodeDropdown(input_plugin='quantumespresso.pw', text="Select code")
number_of_nodes = ipw.IntText(value=1,
                              step=1,description = "that will be run on",
                              disabled=False,
                              layout=ipw.Layout(width="180px"),
                              style={"description_width":"120px"},)
cpus_per_node = ipw.IntText(
    value=1,
    step=1,
    description = "",
    disabled=False,
    layout=ipw.Layout(width="50px"),
    style={"description_width":"0px"},
)
structure_widget = StructureManagerWidget(
    importers=[
        ("From computer", StructureUploadWidget()),
        ("COD", CodQueryWidget()),
        ("AiiDA database", StructureBrowserWidget()),
        ("SMILES", SmilesWidget()),
        ("From Examples", StructureExamplesWidget(
            examples=[
                ("Silicon oxide", "miscellaneous/structures/SiO2.xyz"),
                ('GaAs','miscellaneous/structures/GaAs.xyz'),
            ]
        )
        ),
    ],
    storable=False,
    node_class='StructureData'
)

def change_submit_calc_visibility(c):
    submit_out.layout.visibility = 'visible' if c['new'] else 'hidden'
structure_widget.observe(change_submit_calc_visibility, names=['has_structure'])

In [ ]:
def setup_calc():
    if code_group.selected_code is None:
        print ("Please select a code")
        return None
    
    if 'optimized_structure' not in globals() and not structure_widget.structure_node:
        print ("Please select a structure")
        return None 
    builder = WorkflowFactory(run_type.value).get_builder()
    params = Dict(dict={
            'SYSTEM': {
                'ecutwfc': 50.,
                'ecutrho': 200.,
            },
        })
    options = {
                'max_wallclock_seconds': 3600*2,
                'resources':{
                    'num_machines': number_of_nodes.value,
                    'num_mpiprocs_per_machine': cpus_per_node.value,
                }
            }
    if 'optimized_structure' in globals():
        structure = optimized_structure
    else:
        structure = structure_widget.structure_node

    if run_type.value == 'quantumespresso.pw.base':
        builder.pw.code = code_group.selected_code
        builder.pw.parameters = params
        builder.pw.metadata.options = options
        builder.kpoints_distance = Float(0.8)
        builder.pseudo_family = Str(pseudo_family.value)
        builder.pw.structure = structure

    elif run_type.value == 'quantumespresso.pw.relax':
        builder.base.pw.code = code_group.selected_code
        builder.base.pw.parameters = params
        builder.base.pw.metadata.options = options
        builder.base.kpoints_distance = Float(0.8)
        builder.base.pseudo_family = Str(pseudo_family.value)
        builder.structure = structure

    elif run_type.value == 'quantumespresso.pw.bands':
        builder.scf.pw.code = code_group.selected_code
        builder.bands.pw.code = code_group.selected_code
        builder.scf.pw.parameters = params
        builder.bands.pw.parameters = params
        builder.scf.pw.metadata.options = options
        builder.bands.pw.metadata.options = options
        builder.scf.kpoints_distance = Float(0.8)
        builder.scf.pseudo_family = Str(pseudo_family.value)
        builder.bands.pseudo_family = Str(pseudo_family.value)
        builder.structure = structure

    return builder

In [ ]:
def submit_process(b):
    global process_outputs
    
    builder = setup_calc()
    if builder:
        process = submit(builder)
        process_node = load_node(process.id)
        follower = ProcessFollowerWidget(process, followers=[RunningCalcJobOutputWidget, ProgressBarWidget])
        export_btn = ExportButtonWidget(process=process)
        display(export_btn, follower)
        follower.follow()
        if 'output_structure' in process_node.outputs:
            global optimized_structure
            optimized_structure = process_node.outputs.output_structure
        show_results(process_node.outputs)

In [ ]:
def process_settings():
    global run_type, pseudo_family
    run_type = ipw.ToggleButtons(
        options=[
            ('scf', 'quantumespresso.pw.base'),
            ('relax', 'quantumespresso.pw.relax'),
            ('bands', 'quantumespresso.pw.bands'),
        ],
        description='Calculation type:',
        style = {'description_width': 'initial'},
    )
    pseudo_family = ipw.ToggleButtons(
        options = {
            'SSSP efficiency': 'SSSP_1.1_efficiency',
            'SSSP accuracy': 'SSSP_1.1_precision',
        },
        description='Pseudopotential family:',
        style = {'description_width': 'initial'},
    )
    
    display(arrow_down,
            ipw.HBox([code_group, number_of_nodes, ipw.HTML("node(s)"),
                      cpus_per_node, ipw.HTML("CPU each")]),
            run_type,
            pseudo_family,
            run_btn)

In [ ]:
def show_results(outputs):
    for key in ['output_trajectory', 'retrieved', 'output_parameters',
                'output_structure', 'band_structure']:
        if key in outputs:
            display(viewer(outputs[key]))
    process_settings()

In [ ]:
submit_out = ipw.Output(layout={'visibility':'hidden'})
with submit_out:
    run_btn.on_click(submit_process)
    process_settings()
display(structure_widget, submit_out)